In [113]:
#include parent folder
import os, sys, inspect

currentdir = os.path.dirname(
    os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

import time
import timeit
import numpy as np
import pytest
import jax
import jax.numpy as jnp
import functools
import scipy
import pandas as pd

from src.domain import Domain
from src.misc import *

In [114]:
def VLJ1(v, w):   
    r = jnp.linalg.norm(v-w) # euclidian distance between points
    return 4 * (4*pow(1 / r, 12) - 2*pow(1 / r, 6))

def VLJ2(v, w):
    r = jnp.linalg.norm(v-w) # euclidian distance between points
    r6 = pow(1 / r, 6)    # recycle exponents
    return (16*r6*r6 - 8*r6)

def VLJ3(r):
    return 4 * (4*pow(1 / r, 12) - 2*pow(1 / r, 6)) # TODO: numpy pow

VLJ1_jit = jax.jit(VLJ1)
VLJ2_jit = jax.jit(VLJ2)
VLJ3_jit = jax.jit(VLJ3)

def Epot_loops_nojit(pos): 
    # primitive Loop   
    v = to3D(pos)
    E = 0
    for i in range(len(v)):
        for j in range(i + 1, len(v)):
            E += VLJ1(v[i], v[j])    # using jit actually shaved off 20% form runtime  
    return E  

def Epot_loops_VLJ1_jit(v): 
    # primitive Loop   
    E = 0
    for i in range(len(v)):
        for j in range(i + 1, len(v)):
            E += VLJ1_jit(v[i], v[j])  
    return E 


def Epot_loops_VLJ2_jit(v): 
    # primitive Loop   
    E = 0
    for i in range(len(v)):
        for j in range(i + 1, len(v)):
            E += VLJ2_jit(v[i], v[j])   
    return E   

def Epot_broadcasted(x):
    n = len(x)
    broadcasted = x[:, np.newaxis, :] - x   # calculate the distance between all particles
    broadcasted = broadcasted.reshape(n*n,3)   # reshape such that we have a list of distance vectors
    dist = np.linalg.norm(broadcasted, axis=1)  # calculate the euclidian norm of all distances
    dist = dist[dist != 0]    # remove 0s
    E = 0
    for d in dist:
        E += VLJ3(d)

    return E /2

def Epot_broadcasted(x):
    n = len(x)
    broadcasted = x[:, np.newaxis, :] - x   # calculate the distance between all particles
    broadcasted = broadcasted.reshape(n*n,3)   # reshape such that we have a list of distance vectors
    dist = np.linalg.norm(broadcasted, axis=1)  # calculate the euclidian norm of all distances
    dist = dist[dist != 0]    # remove 0s
    E = 0
    for d in dist:
        E += VLJ3(d)

    return E /2


def Epot_broadcasted_jnp(x):
    n = len(x)
    broadcasted = x[:, jnp.newaxis, :] - x   # calculate the distance between all particles
    broadcasted = broadcasted.reshape(n*n,3)   # reshape such that we have a list of distance vectors
    dist = jnp.linalg.norm(broadcasted, axis=1)  # calculate the euclidian norm of all distances
    dist = dist[dist != 0]    # remove 0s
    E = 0
    for d in dist:
        E += VLJ3(d)

    return E /2

def Epot_vectorized(x):
    n = len(x)
    broadcasted = x[:, jnp.newaxis, :] - x   # calculate the distance between all particles
    broadcasted = broadcasted.reshape(n*n,3)   # reshape such that we have a list of distance vectors
    r = jnp.linalg.norm(broadcasted, axis=1)  # calculate the euclidian norm of all distances
    r = r[r != 0]    # remove 0s
    return 4 * (4*np.power(r, -12) - 2*np.power(r, -6)).sum() / 2 # TODO: numpy pow


def Epot_vectorized2(x):
    n = len(x)
    broadcasted = x[:, jnp.newaxis, :] - x   # calculate the distance between all particles while utilizing jnp.broadcast
    broadcasted = broadcasted.reshape(n*n,3)
    r = jnp.linalg.norm(broadcasted, axis=1)  # calculate the euclidian norm of all distances
    r6 =  jnp.nan_to_num(jnp.power(r, -6), posinf=0.0)
    return 4*(2*r6*r6 - r6).sum()

def Epot_vectorized2_dot(x):
    n = len(x)
    broadcasted = x[:, jnp.newaxis, :] - x   # calculate the distance between all particles while utilizing jnp.broadcast
    broadcasted = broadcasted.reshape(n*n,3)
    r = jnp.linalg.norm(broadcasted, axis=1)  # calculate the euclidian norm of all distances
    r6 =  jnp.nan_to_num(jnp.power(r, -6), posinf=0.0)
    return 4*(2*jnp.dot(r6,r6) - r6.sum())


def Epot_vectorized3(x):
    n = len(x)
    broadcasted = x[:, jnp.newaxis, :] - x   # calculate the distance between all particles while utilizing jnp.broadcast
    broadcasted = broadcasted.reshape(n*n,3)
    broadcasted = broadcasted[(n*n)//2:]
    r = jnp.linalg.norm(broadcasted, axis=1)  # calculate the euclidian norm of all distances
    r6 =  jnp.nan_to_num(jnp.power(r, -6), posinf=0.0)
    return 8*(2*r6*r6 - r6).sum()

def Epot_vectorized4(x):
    n2 = len(x)*len(x)
    broadcasted = jnp.array(x[:, jnp.newaxis, :] - x)   # calculate the distance between all particles while utilizing jnp.broadcast
    broadcasted = broadcasted.reshape(n2,3) # to Nx3
    broadcasted = broadcasted[(n2)//2:]
    r = jnp.linalg.norm(broadcasted, axis=1)  # calculate the euclidian norm of all distances
    r6 =  jnp.nan_to_num(jnp.power(r, -6), posinf=0.0)
    return 8*(2*r6*r6 - r6).sum()


def Epot_vectorized_einsum(x):
    n = len(x)
    broadcasted = x[:, jnp.newaxis, :] - x   # calculate the distance between all particles while utilizing jnp.broadcast
    v = broadcasted.reshape(n*n,3)
    r = jnp.einsum("ij,ij->i",v,v)  # calculate squared norm
    r6 =  jnp.nan_to_num(jnp.power(r, -3), posinf=0.0)  # = (1/r²)^6
    return 4*(2*r6*r6 - r6).sum()


def Epot_cdist(pos):
    return np.triu(scipy.spatial.distance.cdist(pos, pos, metric=VLJ1)).sum()

def Epot_cdist_jnp(pos):
    return jnp.triu(scipy.spatial.distance.cdist(pos, pos, metric=VLJ1)).sum()

Epot_loops_jit = jax.jit(Epot_loops_VLJ1_jit)
Epot_loops2_jit = jax.jit(Epot_loops_VLJ2_jit)
Epot_vectorized2_jit = jax.jit(Epot_vectorized2)
Epot_vectorized3_jit = jax.jit(Epot_vectorized3)
Epot_vectorized4_jit = jax.jit(Epot_vectorized4)
Epot_vectorized_einsum_jit = jax.jit(Epot_vectorized_einsum)
Epot_vectorized2_dot_jit = jax.jit(Epot_vectorized2_dot)

In [115]:
def benchMark(domain, Epot_func, label="", n=5):
    result = Epot_func(domain.pos)  # call so that the jit compiler does not affect the benchmark

    timer = timeit.Timer(functools.partial(Epot_func, domain.pos))
    t = timer.timeit(n) / n # Rrepeat n times and take average
    """
    t = 0
    for i in range(n):
        start = time.time()
        Epot_func(domain.pos)
        end = time.time()
        t += (end - start)

    t = t /n
    """

    return pd.DataFrame(data={
        'time': [t], 
        'label': [label], 
        "result" : [result],
        "n_particles" : [len(domain.pos)],
        "size" : [domain.length]
        })

domain = Domain()
domain.fill(10,20,1)
data = pd.DataFrame()

In [116]:
# slow  ones
n = 1
data = data.append(benchMark(domain, Epot_loops_nojit, "primitive loop, no jit", n))
data = data.append(benchMark(domain, Epot_loops_VLJ1_jit, "primitive loop, jit", n))

In [117]:
# fast ones but long compile time
n = 20
data = data.append(benchMark(domain, Epot_loops_jit, "loop, all jitted", n))
data = data.append(benchMark(domain, Epot_loops2_jit, "loops, all jitted, recycled exonents", n))

In [118]:
# broadcasted
n = 10
data = data.append(benchMark(domain, Epot_broadcasted, "broadcasted", n))
data = data.append(benchMark(domain, Epot_broadcasted_jnp, "broadcasted, jnp", n))

In [119]:
#vectocized
n = 100
data = data.append(benchMark(domain, Epot_vectorized, "vectorized", n))
data = data.append(benchMark(domain, Epot_vectorized2, "vectorized, recycled", n))
data = data.append(benchMark(domain, Epot_vectorized3, "vectorized, recycled, half slice", n))
data = data.append(benchMark(domain, Epot_vectorized4, "vectorized, compound, half slice", n))
data = data.append(benchMark(domain, Epot_vectorized_einsum, "vectorized, einsum", n))
data = data.append(benchMark(domain, Epot_vectorized2_jit, "vectorized, recycled, jit", n))
#data = data.append(benchMark(domain, Epot_vectorized3_jit, "vectorized, recycled, half, slice, jit", n))
#data = data.append(benchMark(domain, Epot_vectorized4_jit, "vectorized, compound, half slice, jit", n))
data = data.append(benchMark(domain, Epot_vectorized_einsum_jit, "vectorized, einsum, jit", n))
data = data.append(benchMark(domain, Epot_vectorized2_dot_jit, "vectorized, dot, jit", n))

In [120]:
# results
data = data.drop_duplicates(subset=["label"])
data = data.sort_values(by="time", ascending=False)
data

,time,label,result,n_particles,size
0,0.227394,"primitive loop, no jit",-0.0126299355,10,20
0,0.009372,"primitive loop, jit",-0.0126299355,10,20
0,0.003540,"vectorized, compound, half slice",-0.01384379,10,20
0,0.003324,"broadcasted, jnp",-0.01263,10,20
0,0.002180,"vectorized, einsum",-0.012629934,10,20
0,0.002118,"vectorized, recycled, half slice",-0.01384379,10,20
0,0.002106,"vectorized, recycled",-0.0126299355,10,20
0,0.002096,vectorized,-0.01263,10,20
0,0.000245,broadcasted,-0.01263,10,20
0,0.000047,"loop, all jitted",-0.0126299355,10,20


# Bigger Domain

In [125]:
domain = Domain()
domain.fill(1200,2000,1)
data = pd.DataFrame()
n = 20


data = data.append(benchMark(domain, Epot_vectorized2_jit, "vectorized, recycled, jit", n))
#data = data.append(benchMark(domain, Epot_vectorized3_jit, "vectorized, recycled, half, slice, jit", n))
#data = data.append(benchMark(domain, Epot_vectorized4_jit, "vectorized, compound, half slice, jit", n))
data = data.append(benchMark(domain, Epot_vectorized_einsum_jit, "vectorized, einsum, jit", n))
data = data.append(benchMark(domain, Epot_vectorized2_dot_jit, "vectorized, dot, jit", n))

data = data.drop_duplicates(subset=["label"])
data = data.sort_values(by="time", ascending=False)
data

,time,label,result,n_particles,size
0,0.002953,"vectorized, dot, jit",-4.850268e-06,1200,2000
0,0.001732,"vectorized, einsum, jit",-4.850267e-06,1200,2000
0,0.000078,"vectorized, recycled, jit",-4.850268e-06,1200,2000


In [122]:

def Epot_vectorized_einsum(x):
    n = len(x)
    broadcasted = x[:, jnp.newaxis, :] - x   # calculate the distance between all particles while utilizing jnp.broadcast
    v = broadcasted.reshape(n*n,3)
    r = jnp.einsum("ij,ij->i",v,v)  # calculate squared norm
    r6 =  jnp.nan_to_num(jnp.power(r, -3), posinf=0.0)  # = (1/r²)^6
    return 4*(2*r6*r6 - r6).sum()

domain = Domain()
domain.fill(2,10,1)
print(Epot_vectorized3(domain.pos))
print(Epot_vectorized(domain.pos))

-0.00071735494
-0.0007173549965955317


# detailed analysis of Epot_vectorized2

In [123]:
def Epot_vectorized2(x):
    df = pd.Series()

    start = time.time()
    n = len(x)
    end = time.time()
    df["len"] = end-start

    start = time.time()
    broadcasted = x[:, jnp.newaxis, :] - x   # calculate the distance between all particles while utilizing jnp.broadcast
    end = time.time()
    df["broadcast"] = end-start

    start = time.time()
    broadcasted = broadcasted.reshape(n*n,3)
    end = time.time()
    df["reshape"] = end-start

    start = time.time()
    r = jnp.linalg.norm(broadcasted, axis=1)  # calculate the euclidian norm of all distances
    end = time.time()
    df["norm"] = end-start

    start = time.time()
    r6 =  jnp.nan_to_num(jnp.power(r, -6), posinf=0.0)
    end = time.time()
    df["r6"] = end-start

    start = time.time()
    res = 4*(2*r6*r6 - r6).sum()
    end = time.time()
    df["res"] = end-start
    return df


domain = Domain()
domain.fill(200, 200, 1)
df = pd.DataFrame()
for i in range(100):
    df = df.append(Epot_vectorized2(domain.pos),ignore_index=True)

df.mean()

<ipython-input-123-2986ebbb10a8>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.Series()


broadcast    0.000530
len          0.000001
norm         0.000258
r6           0.001069
res          0.001486
reshape      0.000003
dtype: float64

In [124]:
def Epot_vectorized2(x):
    n = len(x)
    broadcasted = x[:, jnp.newaxis, :] - x   # calculate the distance between all particles while utilizing jnp.broadcast
    broadcasted = broadcasted.reshape(n*n,3)
    r = jnp.linalg.norm(broadcasted, axis=1)  # calculate the euclidian norm of all distances
    r6 =  jnp.nan_to_num(jnp.power(r, -6), posinf=0.0)
    return 4*(2*r6*r6 - r6).sum()

def Epot_vectorized_dot(x):
    n = len(x)
    broadcasted = x[:, jnp.newaxis, :] - x   # calculate the distance between all particles while utilizing jnp.broadcast
    broadcasted = broadcasted.reshape(n*n,3)
    r = jnp.linalg.norm(broadcasted, axis=1)  # calculate the euclidian norm of all distances
    r6 =  jnp.nan_to_num(jnp.power(r, -6), posinf=0.0)
    return 4*(2*jnp.dot(r6,r6) - r6.sum())

domain = Domain()
domain.fill(200, 200, 1)
print(Epot_vectorized2(domain.pos))
print(Epot_vectorized_dot(domain.pos))

-0.041363675
-0.041363675
